# Un ejemplo un poco más complicado

Vamos a hacer un caso muy simplificado de una cuenta de un banco

###  Se importan la librerias de akka 

In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.5.14`

import $ivy.$                                     

## Se define el protocolo de mensajería

### Protocolo de entrada

In [2]:
sealed trait AccountIn

// Commands

sealed trait AccountCommand extends AccountIn { 
    val amount : Int
    assume( amount >= 0, "Amount should not be a negative number" )
}

final case class Withdrawal(amount : Int) extends AccountCommand
final case class Income(amount : Int) extends AccountCommand

// Queries

sealed trait AccountQuery extends AccountIn

final case object GetBalance extends AccountQuery

defined trait AccountIn
defined trait AccountCommand
defined class Withdrawal
defined class Income
defined trait AccountQuery
defined object GetBalance

### Protocolo de salida

In [3]:
sealed trait AccountOut

final case class CurrentBalance( balance: Int ) extends AccountOut
final case class DeltaBalance( delta : Int, error : Option[Throwable] = None ) extends AccountOut

defined trait AccountOut
defined class CurrentBalance
defined class DeltaBalance

### Eventos

In [4]:
sealed trait AccountEvent {
    
    val idAccount : String
    val amount: Int
}

case class WithdrawalCreated( val idAccount: String, val amount : Int ) extends AccountEvent
case class IncomeCreated( val idAccount: String, val amount : Int ) extends AccountEvent

defined trait AccountEvent
defined class WithdrawalCreated
defined class IncomeCreated

---

---
## Se define un actor 'Cuenta'

In [5]:
import akka.actor._
import scala.collection.mutable.Queue
import scala.util._

class ActorAccount( private val id: String, 
                       private val updateBalance : (Int, Int) => Try[Int], 
                       private val queueCQRS: Queue[AccountEvent] ) extends Actor {
    
    var balance : Int = 0
    
    override def receive = {
        
        case command : AccountCommand => manageCommads( command )
        case querry  : AccountQuery   => manageQueries( querry )
        case other                    => unhandled( other )
        
    }
    
    private def manageCommads( command: AccountCommand ) : Unit = {
        
        val amount =  command match {
            case Withdrawal( amount ) => -1 * amount
            case Income( amount ) => amount
        }
        
        updateBalance( amount, balance ) match {            
            case Success( newBalance ) => {
                sender() ! DeltaBalance( newBalance - balance ) 
                sendEvent( command )
                balance = newBalance
            }
            case Failure(  error ) => sender() ! DeltaBalance( 0, Some( error ) )         
        }   
        
    }
    
    private def sendEvent( command: AccountCommand ) {
        
        val event : AccountEvent = command match {
            case Withdrawal( amount ) => WithdrawalCreated( id, amount ) 
            case Income( amount )     => IncomeCreated( id, amount ) 
        }
        
        queueCQRS.enqueue( event )
        
    }
    
    private def manageQueries( queries : AccountQuery ) : Unit = queries match {
        case GetBalance => sender() !  CurrentBalance( balance )
    }
    
    
}

import akka.actor._

import scala.collection.mutable.Queue

import scala.util._


defined class ActorAccount

### Utilidades para manejar cuentas

In [6]:
object Account {
    def apply( account :ActorRef ) = new Account( account )
}

class Account( private val account :ActorRef  ) {
    
 import scala.concurrent._
 import scala.concurrent.duration._
 import akka.pattern._
 import akka.actor
 import akka.util.Timeout
    
 implicit val timeout = Timeout( 5 seconds )

    
 def makeWithdrawal( amount : Int ) : Future[ DeltaBalance ] = {
        ( account ? Withdrawal( amount ) ).mapTo[DeltaBalance]            
  }
    
 def makeIncome( amount : Int ) : Future[DeltaBalance] = {
        ( account ? Income( amount ) ).mapTo[DeltaBalance]            
  }
    
  def getBalance : Future[CurrentBalance] = {
     ( account ? GetBalance ). mapTo[ CurrentBalance ]
 }
    
}



defined object Account
defined class Account

---

----
## Implementaciones

### Lógica de negocio
Se define una lógica de negocio simple. En este caso no se admiten descubiertos, pero por ejemplo se pueden implementar diferentes lógicas como un porcentaje de descubierto dependiendo del balance. 
> El objetivo de esto es que la lógica puede estar separada del actor y puede ser validada y probada aparte

In [7]:
import scala.util._

val updateBalance : (Int,Int) => Try[Int] = ( amount, balance ) => {
   
    val newBalance = amount + balance
    
    if( newBalance >= 0 ) {
    
        Success( newBalance )
        
    } else {
        
        Failure( new IllegalStateException( s"It should not be in red( ${newBalance} )" ) )
    }
    
}

import scala.util._


updateBalance: (Int, Int) => Try[Int] = $sess.cmd6Wrapper$Helper$$Lambda$3294/670360747@1c9cab80

### Indirección de publicación de eventos
Se define una cola que será la indirección de publicación de eventos.
En este caso para esta prueba será una `ConcurrentLinkedQueue`.   
> En un sistema real puede ser un akka stream con su fuente '_materializada_' en una cola

In [8]:
import scala.collection.mutable.Queue

val queueCQRS = Queue[AccountEvent]()

import scala.collection.mutable.Queue


queueCQRS: Queue[AccountEvent] = Queue()

---

---
## Probandolo todo

### _Testing: Utilidades_

In [9]:
object TestUtil {
    
    import scala.concurrent._, duration._
    import akka.pattern._
    import akka.util.Timeout


    val tm = 5 seconds
    implicit val timeout = Timeout( tm )

    def result[T]( future : Future[T] ) = Try {
        Await.result( future, tm )
    }
    
}

defined object TestUtil

### Se crea el sistema de actores

In [10]:
val system = ActorSystem.create( "test-1" )

system: ActorSystem = akka://test-1

### Se crean dos actores cuenta

In [11]:
val accountOneActor = system.actorOf{
        Props( new ActorAccount( "accountOne", updateBalance, queueCQRS ) ) 
}

val accountTwoActor = system.actorOf{
        Props( new ActorAccount( "accountTwo", updateBalance, queueCQRS ) ) 
}

val accountOne = Account( accountOneActor )
val accountTwo = Account( accountTwoActor )

accountOneActor: ActorRef = Actor[akka://test-1/user/$a#480683290]
accountTwoActor: ActorRef = Actor[akka://test-1/user/$b#-1522344440]
accountOne: Account = $sess.cmd5Wrapper$Helper$Account@288842d7
accountTwo: Account = $sess.cmd5Wrapper$Helper$Account@b8fb511

### Se hace un ingreso incial a las dos cuentas

> Aquí se hace `Await` sólo por motivos de testing

In [12]:

TestUtil.result{
    accountOne.makeIncome( 1000 ) 
}

TestUtil.result{
    accountTwo.makeIncome( 1000 ) 
}


res11_0: Try[DeltaBalance] = Success(DeltaBalance(1000, None))
res11_1: Try[DeltaBalance] = Success(DeltaBalance(1000, None))

#### Se comprueba los balances

> `Await` sólo por motivos de testing

In [13]:
TestUtil.result{
    accountOne.getBalance 
}

TestUtil.result { 
    accountTwo.getBalance 
}

res12_0: Try[CurrentBalance] = Success(CurrentBalance(1000))
res12_1: Try[CurrentBalance] = Success(CurrentBalance(1000))

#### Se comprueban los eventos

In [14]:
queueCQRS.toList ; queueCQRS.clear

res13_0: List[AccountEvent] = List(IncomeCreated(accountOne,1000), IncomeCreated(accountTwo,1000))

### Se hace una transferencia

Se simula una operación/compensacion siguiendo el patrón sagas

In [15]:


object Transfer {
    
    import scala.concurrent.ExecutionContext.Implicits.global
    import scala.concurrent._
    
    def transfer( from : Account, to: Account ) ( amount : Int ) = {
        
        val withdrawn = from.makeWithdrawal( amount )
        
        val income = to.makeIncome( amount )        

        for {
             w    <- withdrawn
             i    <- income
             comp <- ( w.delta + i.delta ) match {
                    case a if ( a < 0 ) => from.makeIncome( amount )
                    case a if ( a > 0 ) => to.makeWithdrawal( amount )
                    case _ => Future{ DeltaBalance(0) }
                }
        } yield {
            ( w, i, comp )
        }
        
    }
    
}



defined object Transfer

In [16]:
val transfersOneToTwo =  Transfer.transfer( accountOne, accountTwo)( _ ) 
val transfersTwoToOne =  Transfer.transfer( accountTwo, accountOne)( _ ) 

transfersOneToTwo: Int => concurrent.Future[(DeltaBalance, DeltaBalance, DeltaBalance)] = $sess.cmd15Wrapper$Helper$$Lambda$3669/916654241@6b909c26
transfersTwoToOne: Int => concurrent.Future[(DeltaBalance, DeltaBalance, DeltaBalance)] = $sess.cmd15Wrapper$Helper$$Lambda$3670/2137939645@6dd62807

## _Probando, probando_

In [19]:
TestUtil.result{
        transfersOneToTwo( 500 ) 
}

res18: Try[(DeltaBalance, DeltaBalance, DeltaBalance)] = Success(
  (DeltaBalance(-500, None), DeltaBalance(500, None), DeltaBalance(0, None))
)

#### Se vuelen a compruebar los balances

> `Await` sólo por motivos de testing

In [20]:
TestUtil.result{
    accountOne.getBalance 
}


TestUtil.result { 
    accountTwo.getBalance 
}

res19_0: Try[CurrentBalance] = Success(CurrentBalance(500))
res19_1: Try[CurrentBalance] = Success(CurrentBalance(1500))

#### Se vuelven a comprobar los eventos

In [21]:
queueCQRS.toList ; queueCQRS.clear

res20_0: List[AccountEvent] = List(
  IncomeCreated(accountTwo,5000),
  WithdrawalCreated(accountTwo,5000),
  WithdrawalCreated(accountOne,500),
  IncomeCreated(accountTwo,500)
)